# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921696819843                   -0.69    5.8         
  2   -7.926167336606       -2.35       -1.22    1.0    261ms
  3   -7.926836313294       -3.17       -2.37    1.6    235ms
  4   -7.926861485390       -4.60       -3.01    2.9    264ms
  5   -7.926861633226       -6.83       -3.35    2.1    267ms
  6   -7.926861667312       -7.47       -3.73    1.5    209ms
  7   -7.926861680448       -7.88       -4.34    1.2    210ms
  8   -7.926861681751       -8.89       -4.81    2.2    235ms
  9   -7.926861681865       -9.95       -5.37    1.4    246ms
 10   -7.926861681872      -11.14       -6.01    2.0    231ms
 11   -7.926861681873      -12.12       -6.67    2.2    242ms
 12   -7.926861681873      -13.41       -7.29    2.0    242ms
 13   -7.926861681873   +    -Inf       -8.25    2.4    250ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921685012549                   -0.69    5.9         
  2   -7.926161117758       -2.35       -1.22    1.0    221ms
  3   -7.926836460875       -3.17       -2.37    1.8    247ms
  4   -7.926861513431       -4.60       -3.02    2.6    256ms
  5   -7.926861633422       -6.92       -3.35    1.9    265ms
  6   -7.926861666604       -7.48       -3.72    1.5    209ms
  7   -7.926861680582       -7.85       -4.40    1.2    204ms
  8   -7.926861681811       -8.91       -4.95    2.1    235ms
  9   -7.926861681860      -10.31       -5.25    1.8    249ms
 10   -7.926861681872      -10.95       -6.00    1.5    217ms
 11   -7.926861681873      -12.06       -7.00    2.2    241ms
 12   -7.926861681873      -13.77       -7.40    3.0    282ms
 13   -7.926861681873      -14.75       -7.98    1.5    211ms
 14   -7.926861681873      -15.05       -8.82    2.9    250ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921707487887                   -0.69    5.8         
  2   -7.926171519133       -2.35       -1.22    1.0    220ms
  3   -7.926839764023       -3.18       -2.37    1.8    296ms
  4   -7.926864928411       -4.60       -3.00    2.8    271ms
  5   -7.926865035486       -6.97       -3.30    2.0    235ms
  6   -7.926865076416       -7.39       -3.70    1.1    206ms
  7   -7.926865091795       -7.81       -4.42    1.4    243ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
